In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/02/25 06:08:39 WARN Utils: Your hostname, dengine resolves to a loopback address: 127.0.1.1; using 192.168.1.221 instead (on interface ens18)
23/02/25 06:08:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/25 06:08:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [19]:
df_green.registerTempTable('green')

In [20]:
df_green_revenue = spark.sql("""
SELECT 
   date_trunc('hour', lpep_pickup_datetime) AS hour, 
   PULocationID AS zone,
   
   SUM(total_amount) AS amount,
   COUNT(1) as number_records

FROM green
WHERE 
    lpep_pickup_datetime >= '2020-01-01'
GROUP BY 
   1, 2
""")

In [21]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-04 18:00:00| 244|308.65000000000003|            13|
|2020-01-19 18:00:00|  41| 578.6900000000002|            46|
|2020-01-20 14:00:00| 166|            292.36|            21|
|2020-01-07 18:00:00| 181|            297.29|            20|
|2020-01-01 04:00:00| 181|200.82999999999998|            11|
|2020-01-01 00:00:00| 255| 666.3399999999999|            28|
|2020-01-23 22:00:00| 129|            272.85|            15|
|2020-01-20 00:00:00| 223|             21.07|             3|
|2020-01-26 01:00:00| 136|             65.22|             2|
|2020-01-20 01:00:00|  82|            137.16|            13|
|2020-01-25 20:00:00|  95|254.22000000000008|            20|
|2020-01-30 07:00:00| 197|             88.41|             2|
|2020-01-28 13:00:00| 134|            189.11|             7|
|2020-01-09 22:00:00|  9

In [15]:
df_green_revenue.write.parquet('data/report/revenue/green', mode='overwrite')

In [22]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [23]:
df_yellow.registerTempTable('yellow')

In [24]:
df_yellow_revenue = spark.sql("""
SELECT 
   date_trunc('hour', tpep_pickup_datetime) AS hour, 
   PULocationID AS zone,
   
   SUM(total_amount) AS amount,
   COUNT(1) as number_records

FROM yellow
WHERE 
    tpep_pickup_datetime >= '2020-01-01'
GROUP BY 
   1, 2
""")

In [25]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-17 13:00:00| 170|6521.6300000000065|           378|
|2020-01-12 23:00:00| 163| 2102.039999999999|           121|
|2020-01-06 16:00:00| 158|           1200.04|            64|
|2020-01-24 22:00:00| 162| 5413.190000000002|           351|
|2020-01-19 19:00:00| 107| 3034.139999999998|           206|
|2020-01-06 13:00:00|  13|1757.1099999999992|            69|
|2020-01-22 10:00:00| 230| 5721.720000000005|           285|
|2020-01-03 12:00:00| 142| 4554.979999999998|           326|
|2020-01-08 02:00:00| 249|1197.7299999999998|            60|
|2020-01-12 16:00:00| 237| 7384.390000000014|           525|
|2020-01-15 08:00:00|  13| 3703.489999999997|           196|
|2020-01-07 12:00:00| 229| 3209.279999999996|           209|
|2020-01-08 16:00:00| 140| 5762.140000000005|           341|
|2020-01-01 17:00:00| 23

In [26]:
df_yellow_revenue.write.parquet('data/report/revenue/yellow', mode='overwrite')

In [29]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')


In [30]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [31]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|            220.21|                    8|
|2020-01-01 00:00:00|  32| 68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  51|              17.8|                   2|              31.0|                    1|
|2020-01-01 00:00:00|  71|              23.8|                   1|              null|                 null|
|2020-01-01 00:00:00|  79|              null|                null|12573.810000000027|                  721|
|2020-01-01 00:00:00|  83| 94.09999999999998|                   7|               9.8|                    1|
|2020-01-01 00:00:00|  85|  

In [33]:
df_join.write.parquet('data/report/revenue/total')

In [35]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [36]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [38]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [39]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [41]:
df_join = spark.read.parquet('data/report/revenue/total')

In [43]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [44]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  13|              null|                null|1214.7999999999997|                   56|
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|
|2020-01-01 00:00:00|  24|              87.6|                   3| 754.9500000000003|                   45|
|2020-01-01 00:00:00|  29|              61.3|                   1|              null|                 null|
|2020-01-01 00:00:00|  33|            317.27|                  11|            255.56|                    8|
|2020-01-01 00:00:00|  41|13

In [45]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2023-02-25 06:57:12--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.129.208, 52.216.78.150, 52.216.152.110, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.129.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0.001s  

2023-02-25 06:57:13 (21.7 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [46]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [47]:
df.write.parquet('zones')

In [48]:
df_zones = spark.read.parquet('zones/')

In [49]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [51]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [52]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  13|              null|                null|1214.7999999999997|                   56|        13|Manhattan|   Battery Park City| Yellow Zone|
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|        14| Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|        15|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00| 

In [53]:
df_result.drop('LocationID').show()

+-------------------+----+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|2020-01-01 00:00:00|  13|              null|                null|1214.7999999999997|                   56|Manhattan|   Battery Park City| Yellow Zone|
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1| Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00|  24|              87.6|                   3| 754.9500000000003|   

In [55]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')